# Part 1 - Scraping

In [ ]:
#Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
import pprint 
from IPython.display import Markdown, display

## Mac Users

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# connect to NASA Mars news Site
url = 'https://redplanetscience.com/'
# Retrieve page with the requests module
browser.visit(url)

In [ ]:
response = requests.get(url)
response

### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

# Examine the results, determine elements that contains sought info.
print(soup.prettify())

In [ ]:
# Test: Result for an article from website
news_title = soup.find('div', class_='content_title').a.text.strip()
news_para = soup.find('div', class_='rollover_description_inner').text.strip()
print(f'------------------------------------------------')
print(f'TITLE: {news_title}')
print(f'------------------------------------------------')
print(f'PARAGRAPH: {news_para}')

### JPL Mars Space Images - Featured Image

In [ ]:
# Visit the url for JPL Featured Space Image
# Set URL
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [ ]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
featured_image = browser.find_by_id("full_image")
featured_image.click()

In [ ]:
# Need more info to find image url
browser.is_element_present_by_text("more info", wait_time=1)
more_info_button = browser.links.find_by_partial_text("more info")
more_info_button.click()

In [ ]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Assign the url string to a variable called `featured_image_url`
# Make sure to find the image url to the full size `.jpg` image
featured_image_url = image_soup.select_one("figure.lede a img").get("src")
featured_image_url

In [ ]:
# Make sure to save a complete url string for this image
featured_image_url = f"https://www.jpl.nasa.gov{featured_image_url}"
print(f"Complete URL String:\n{featured_image_url}")

### Mars Facts

In [ ]:
# Visit the Mars Facts webpage
# Set URL
url = 'https://galaxyfacts-mars.com/'
browser.visit(url)

In [ ]:
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_facts_df)

In [ ]:
# Clean up DataFrame, set index
mars_facts_df.columns=["Planet Profile", "Value"]
mars_facts_df.set_index("Planet Profile", inplace=True)
mars_facts_df

### Mars Hemispheres

In [ ]:
# Visit the Astrogeology site
# Set URL
url = 'https://marshemispheres.com/'
browser.visit(url)

In [ ]:
# Parse Results HTML with BeautifulSoup
html = browser.html
mars_weather_soup = BeautifulSoup(html, "html.parser")

In [ ]:
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name
hemisphere_img_urls = []

links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[item].click()
    
    sample_img = browser.links.find_by_text("Sample").first
    
    hemisphere["title"] = browser.find_by_css("h2.title").text
    hemisphere["img_url"] = sample_img["href"]     
    
    hemisphere_img_urls.append(hemisphere)
    
    browser.back()

In [ ]:
# Use a Python dictionary to store the data using the keys `img_url` and `title`
hemisphere_img_urls